In [1]:
import anndata
import scvelo as scv
import numpy as np
from deepvelo.utils import velocity, velocity_confidence, latent_time, update_dict
from deepvelo.utils.preprocess import autoset_coeff_s
from deepvelo.utils.plot import statplot, compare_plot
from deepvelo import train, Constants
import os
import time

Using backend: pytorch


In [2]:
dataset = "Pancreas"
root = '/scratch/blaauw_root/blaauw1/gyichen'
adata = anndata.read_h5ad(f'{root}/data/{dataset}_pp.h5ad')

In [3]:
save_dir = f"{root}/data/deepvelo"
configs = {
    "name": "DeepVelo-Pancreas", # name of the experiment
    "arch":{"args":{"pred_unspliced":True}},
    "loss": {"args": {"coeff_s": autoset_coeff_s(adata)}},
    "trainer": {"verbosity": 0, "save_dir": save_dir}, # increase verbosity to show training progress
}
configs = update_dict(Constants.default_configs, configs)

The ratio of spliced reads is 83.2% (between 70% and 85%). Suggest using coeff_s 0.75.


In [4]:
t_start = time.time()
velocity(adata, mask_zero=False)
trainer = train(adata, configs)
run_time = time.time() - t_start

computing velocities
    finished (0:00:01) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
building graph


INFO:train:Beginning training of DeepVelo-Pancreas ...


velo data shape: torch.Size([3696, 2000])
velo_mat shape: (3696, 2000)
--> added 'velocity' (adata.layers)
--> added 'velocity_unspliced' (adata.layers)
--> added 'cell_specific_beta' (adata.layers)
--> added 'cell_specific_gamma' (adata.layers)
--> added 'cell_specific_alpha' (adata.layers)


In [6]:
latent_time(adata)
adata.obs['dv_time'] = adata.obs['latent_time'].to_numpy()
del adata.obs['latent_time']
adata.uns['dv_run_time']=run_time
os.makedirs(save_dir, exist_ok=True)
adata.write_h5ad(f'{save_dir}/{dataset}.h5ad')
print(f'Total run time: {run_time}')

computing latent time using root_cells as prior
    finished (0:00:00) --> added 
    'latent_time', shared time (adata.obs)
Total run time: 129.47484827041626
